# GAE Link Prediction Results & Evaluation

This notebook demonstrates the GitHub Collaboration Graph Autoencoder (GAE) pipeline with:
1. **Metrics comparison**: Baselines vs GAE
2. **Interactive visualization** of top predicted links
3. **Training analysis**: Loss curves, embedding distributions
4. **Reproducibility tips** and next steps

## Setup: Load Results

First, load the outputs from the pipeline and GAE training.

In [ ]:
import pandas as pd
import numpy as np
import json
import os
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Configuration
DATA_ROOT = "data"  # Change to "notebooks/data" if running from repo root
PROCESSED_DIR = os.path.join(DATA_ROOT, "processed")

# Load key files
baseline_metrics_path = os.path.join(PROCESSED_DIR, "baseline_metrics.json")
gae_metrics_path = os.path.join(PROCESSED_DIR, "gae_metrics.json")
gae_logs_path = os.path.join(PROCESSED_DIR, "gae_training_logs.json")
predictions_path = os.path.join(PROCESSED_DIR, "predicted_links_top50.csv")
nodes_path = os.path.join(PROCESSED_DIR, "nodes.csv")

# Load JSON files
with open(baseline_metrics_path) as f:
    baseline_metrics = json.load(f)
    
with open(gae_metrics_path) as f:
    gae_metrics = json.load(f)
    
with open(gae_logs_path) as f:
    gae_logs = json.load(f)

# Load CSVs
predictions_df = pd.read_csv(predictions_path)
nodes_df = pd.read_csv(nodes_path)

print(f"✓ Loaded baseline metrics")
print(f"✓ Loaded GAE metrics: AUC={gae_metrics['auc']:.4f}, AP={gae_metrics['ap']:.4f}")
print(f"✓ Loaded {len(predictions_df)} predicted links")
print(f"✓ Loaded {len(nodes_df)} nodes")

## 1. Metrics Comparison: Baselines vs GAE

In [ ]:
# Extract baseline metrics
baseline_list = baseline_metrics["baselines"]
baseline_df = pd.DataFrame([
    {
        "Method": m["method"],
        "AUC": m["auc"],
        "AP": m["ap"]
    }
    for m in baseline_list
])

# Add GAE results
gae_row = pd.DataFrame([{
    "Method": "GAE",
    "AUC": gae_metrics["auc"],
    "AP": gae_metrics["ap"]
}])

# Combine
comparison_df = pd.concat([baseline_df, gae_row], ignore_index=True)

print("\n📊 Link Prediction Evaluation Results:")
print(comparison_df.to_string(index=False))

# Visualization
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=comparison_df["Method"],
    y=comparison_df["AUC"],
    mode="markers+lines",
    name="AUC",
    marker=dict(size=12, color="steelblue"),
    line=dict(color="steelblue")
))

fig.add_trace(go.Scatter(
    x=comparison_df["Method"],
    y=comparison_df["AP"],
    mode="markers+lines",
    name="AP",
    marker=dict(size=12, color="coral"),
    line=dict(color="coral")
))

fig.update_layout(
    title="Link Prediction: Baselines vs GAE",
    xaxis_title="Method",
    yaxis_title="Score",
    hovermode="x unified",
    height=500,
    template="plotly_white"
)

fig.show()

# Summary
print(f"\n✨ GAE improvement over best baseline:")
best_baseline_auc = baseline_df["AUC"].max()
best_baseline_ap = baseline_df["AP"].max()
gae_auc_gain = (gae_metrics["auc"] - best_baseline_auc) / best_baseline_auc * 100
gae_ap_gain = (gae_metrics["ap"] - best_baseline_ap) / best_baseline_ap * 100
print(f"  AUC: +{gae_auc_gain:.1f}%")
print(f"  AP: +{gae_ap_gain:.1f}%")

## 2. Top 50 Predicted Links

In [ ]:
# Display top predictions
print("🔗 Top 20 Predicted Future Collaborations:")
print(predictions_df[["u", "v", "score"]].head(20).to_string(index=False))

# Interactive table
fig_table = go.Figure(data=[go.Table(
    header=dict(
        values=["Rank", "Developer 1", "Developer 2", "Prediction Score"],
        fill_color="steelblue",
        align="left",
        font=dict(color="white", size=12)
    ),
    cells=dict(
        values=[
            np.arange(1, len(predictions_df) + 1),
            predictions_df["u"],
            predictions_df["v"],
            predictions_df["score"].round(4)
        ],
        fill_color="lavender",
        align="left"
    )
)])

fig_table.update_layout(
    title="Top 50 Predicted Developer Pairs",
    height=600
)

fig_table.show()

# Score distribution
fig_dist = px.histogram(
    predictions_df,
    x="score",
    nbins=20,
    title="Prediction Score Distribution",
    labels={"score": "GAE Score", "count": "Frequency"},
    color_discrete_sequence=["steelblue"]
)

fig_dist.update_layout(height=400, template="plotly_white")
fig_dist.show()

print(f"\nScore statistics:")
print(f"  Min: {predictions_df['score'].min():.4f}")
print(f"  Max: {predictions_df['score'].max():.4f}")
print(f"  Mean: {predictions_df['score'].mean():.4f}")
print(f"  Median: {predictions_df['score'].median():.4f}")

## 3. Training Analysis: Loss Curves

In [ ]:
# Extract training logs
logs_df = pd.DataFrame(gae_logs)

# Plot loss curve
fig_loss = px.line(
    logs_df,
    x="epoch",
    y="loss",
    title="GAE Training Loss Over Epochs",
    labels={"epoch": "Epoch", "loss": "Reconstruction Loss"},
    markers=True
)

fig_loss.update_layout(
    height=400,
    template="plotly_white",
    hovermode="x"
)

fig_loss.show()

print(f"\n📈 Training Summary:")
print(f"  Total epochs: {gae_metrics['epochs']}")
print(f"  Initial loss: {logs_df['loss'].iloc[0]:.4f}")
print(f"  Final loss: {logs_df['loss'].iloc[-1]:.4f}")
print(f"  Loss reduction: {(1 - logs_df['loss'].iloc[-1]/logs_df['loss'].iloc[0])*100:.1f}%")
print(f"  Device: {gae_metrics['device']}")
print(f"  Timestamp: {gae_metrics['timestamp']}")

## 4. Next Steps & Recommendations

### What the Results Show

- **GAE outperforms baselines** by learning complex edge patterns from graph topology and node features
- **Top 50 predictions** represent the most likely future collaborations based on embedding proximity
- **Training convergence** (loss curve) shows the model learned meaningful representations

### Recommended Next Steps

1. **Temporal Validation** (High Priority)
   - Implement time-based train/test split using commit timestamps
   - Train on first 80% of commits, test on last 20% (by date)
   - More realistic evaluation of prediction capability

2. **Enhanced Node Features** (Medium Priority)
   ```python
   # In prepare_pyg_data.py, add:
   pagerank = nx.pagerank(G)
   clustering = nx.clustering(G)
   # Concatenate to X before scaling
   ```

3. **Validation Workflow**
   - Run baseline with: `python scripts/baselines_link_pred.py --data-root data`
   - Run GAE with: `python scripts/train_gae.py --data-root data --sample`
   - Compare metrics in this notebook

4. **Cross-Validation** (Advanced)
   - Train multiple GAE models with different seeds (see `--seed` flag)
   - Report mean ± std of AUC/AP
   - More robust evaluation

5. **Production Deployment**
   - Save the best model: `gae_model.pt`
   - Load & apply to new developers: use embeddings for similarity matching
   - Monitor prediction quality over time

### Reproducibility Checklist

✅ All scripts use `--seed` for deterministic results  
✅ PyTorch/PyG versions pinned in `requirements.txt`  
✅ Training logs saved to `gae_training_logs.json`  
✅ Model weights saved to `gae_model.pt`  
✅ Embeddings saved to `gae_embeddings.npy`  
✅ Metrics and predictions saved to JSON/CSV for inspection